In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps
import KY_utils.KY_Corel as Corel

# restore saved variables
%store -r summary_violent6_ky_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_violence_six_month'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_violence_six_month'].values

In [3]:
#### CART
depth = [6,7]
cart_summary = advance.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [20]
depth = [2]
learning_rate = [0.5]
ebm_summary = advance.EBM(KY_X, KY_Y, FL_X, FL_Y, 
                          learning_rate=learning_rate, 
                          depth=depth, 
                          estimators=estimators, 
                          seed=816)

In [5]:
cart_summary, ebm_summary

({'best_auc': 0.8613072511206804,
  'auc_diff': 0.013243636104320222,
  'best_param': {'max_depth': 6},
  'FL_score': 0.5909209478801429},
 {'best_auc': 0.8586676079188744,
  'auc_diff': 0.004598075732093587,
  'best_param': {'learning_rate': 0.5,
   'max_tree_splits': 2,
   'n_estimators': 20},
  'FL_score': 0.6250588613920084})

### Lasso Stumps

In [6]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence>=1']
KY_Y = KY_stumps['recid_violence_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence>=1']
FL_Y = FL_stumps['recid_violence_six_month'].values

## columns 
cols = KY_X.columns

In [7]:
c_grid = {'C': [0.03, 0.05, 0.07]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [8]:
stumps_summary

{'best_auc': 0.8695419474228788,
 'best_params': {'C': 0.03},
 'auc_diffs': 0.006574104594489927,
 'FL_score': 0.6345088673204939}

### RiskSLIM

In [9]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.03, random_state=816).fit(KY_X, KY_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

65

In [10]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [11]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'general2', 
                                max_coef = 20, 
                                max_coef_number = 10, 
                                max_runtime=400, 
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
09/17/19 @ 09:32 PM | 2701 rows in lookup table
09/17/19 @ 09:32 PM | ------------------------------------------------------------
09/17/19 @ 09:32 PM | runnning initialization procedure
09/17/19 @ 09:32 PM | ------------------------------------------------------------
09/17/19 @ 09:32 PM | CPA produced 2 cuts
09/17/19 @ 09:32 PM | running naive rounding on 99 solutions
09/17/19 @ 09:32 PM | best objective value: 0.0911
09/17/19 @ 09:32 PM | rounding produced 5 integer solutions
09/17/19 @ 09:32 PM | best objective value is 0.0965
09/17/19 @ 09:32 PM | running sequential rounding on 99 solutions
09/17/19 @ 09:32 PM | best objective value: 0.0911
09/17/19 @ 09:32 PM | sequential rounding produced 6 integer solutions
09/17/19 @ 09:32 PM | best objective value: 0.0944
09/17/19 @ 09:32 PM | polishing 11 solutions
09/17/19 @ 09:32 PM | best objective value: 0.0944
09/17/19 @ 09:32 PM | polishing produced 4 integer solutions
09/17/19

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
09/17/19 @ 09:32 PM | adding 429 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0936.
Tried aggregator 1 time.
Reduced MIP has 67 rows, 134 columns, and 263 nonzeros.
Reduced MIP has 65 binaries, 67 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.16 ticks)
Probing time = 0.00 sec. (0.07 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.08 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Varia

In [12]:
riskslim_summary

{'FL_Score': 0.5935311753646019}

### Corel

In [2]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence>=1']
KY_Y = KY_stumps['recid_violence_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence>=1']
FL_Y = FL_stumps['recid_violence_six_month'].values

In [3]:
KY_score, FL_score = Corel.corel_cv(KY_X, KY_Y, FL_X, FL_Y, max_card=2, c=1e-3, seed=816)

In [17]:
#### save results
summary_violent6_ky_inter_model = {"CART": cart_summary,
                                   "EBM": ebm_summary,
                                   "Stumps": stumps_summary,
                                   "RiskSLIM": riskslim_summary, 
                                   "Corel": FL_score}
%store summary_violent6_ky_inter_model

Stored 'summary_general2_ky_inter_model' (dict)


In [15]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Six Month\\"
results = [["Violent",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_Score'], 3), 
            round(Fl_score, 3)]]
with open(path + 'Six Month KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)